# Introdução

O objetivo deste notebook é estudar alguns modelos de linguagem e como usar expressões regulares no NLP. A base de dados que vou utilizar é a do [stackoverflow](https://stackoverflow.com/) com textos em português, inglês e espanhol.

## Importações

In [1]:
import pandas as pd
import numpy as np

In [2]:
stackoverflow_portugues = pd.read_csv('dados/stackoverflow_portugues.csv')
stackoverflow_ingles = pd.read_csv('dados/stackoverflow_ingles.csv')
stackoverflow_espanhol = pd.read_csv('dados/stackoverflow_espanhol.csv')

In [3]:
stackoverflow_portugues.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191


Acima temos os 5 primeiros registros da base de dados do stackoverflow. São as perguntas com maiores pontuações e mais visualizações. As variáveis são:

* Id: Identificador da questão.
* Título: Titulo da questão
* Questão: A pergunta efetivamente
* Tags: As tags relacionadas à quetão
* Pontuação: Pontuação da questão
* Visualizações: nº de visualizações

#### Exemplo de questão

Vamos verificar qual conteúdo é armazenado na variável `Questão`

In [4]:
print(stackoverflow_portugues['Questão'][5])

<p>Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:</p>

<pre><code>//----CONSULTA SQL----//
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');
</code></pre>

<p>Logo, digamos que o usuário usar a sentença: <code>1); DROP TABLE Produtos;</code> para ao campo <code>valor</code> o comando ficaria: </p>

<pre><code>insert into Produtos (coluna) values(1); DROP TABLE Produtos;
</code></pre>

<p>Ele vai inserir um novo registro cujo o campo <code>coluna</code> será <code>1</code> e logo em seguida ele vai deletar a tabela Produtos.</p>

<p>Como posso melhorar meu código para prevenir essa situação?</p>



Esse é um exemplo de dados em português que será usado para treinar um modelo de identificação de idiomas. Um ponto importante é que não se tem só texto, também tem tags html então é preciso retirar estas tags. Outra estrutura de texto que tem no dado são códigos por exemplo: 

```sql
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');
```

é preciso retirar também esse tipo de estrutura do dado, isto porque a maioria das linguagens de programação e marcação é em inglês e isso pode gerar um viés do modelo classificar todos os textos com trechos em códigos como um idioma inglês.

Além disso a pontuação também precisa ser removida. Tomando a oração como exemplo: 

> Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL.

se queremos identificar o idioma em português não faz sentido deixar virgulas e pontos finais.

# REGEX

Como precisamos fazer várias filtragens em dados textuais, uma forma bem usual de fazer este ofício é através das expressões regulares, ou REGEX. REGEX é similar á uma linguagem de marcação (mas não é) dentro de uma própria linguagem, tipo Java, C ou python, que permite identificar padrões em estruturas de caracters.

Por exemplo, imagine que temos a tarefa de identificar um provedor de email e temos uma lista de emaisl:
```
joao_miguel@gmail.com
maria_eduarda@hotmail.com.br
alice_124@bol.com
```
como fazer isso? Bom, é facil ver que o provedor de email tem um padrão no email em si. Geralmente o provedor precede o ($@$) e antecede o ($\cdot$), ou seja, dado este padrão que identificamos podemos filtrar usando expressão regular. Um excelente site para o estudo de expressões regulares é o https://regex101.com/

Para fazer esta filtragem utilizamos os chamados [*Meta Caracteres*](https://tdn.engpro.totvs.com.br/display/tec/RegEx+-+Metacaracteres), que são marcações utilizadas para seleção de certas estruturas num texto. Por exemplo, para captar todos os provedores de email podemos usar a seguinte expressão regular:


> `@[a-z]+\.` se lê como: @[todos alpha] com mais de 1 caracter até o \. 

se desejar-mos limitar o provedores com no máximo 5 caracteres substituimos o ($+$) por $\{5\}$

> `@[a-z]{1,5}\.` se lê como: @[todos alpha] com no minimo 1 até 5 caracter até o \. 

Outro meta caracter muito importante é o circunflexo `(^)`, ele é uma forma de obrigar uma estrutura de caracters a iniciar com uma letra, por exemplo, numa regex da forma `^j.*` aplicada a nossa lista de emails, vai tornar apenas `joao_miguel@gmail.com` porque é o único que começa com a letra *j*.

É possível criar grupos de seleção, por exemplo, na expressão regular `^(j|a).*` aplicada a nossa lista de email, vai retornar `joao_miguel@gmail.com` e `alice_124@bol.com`, porque são os dois emails que começam com *j* ou *a*

Da mesma forma que o `(^)` para as estruturas que começam obrigatoriamente com os caracteres, temos uma forma de fazer com que a string termine obrigatoriamente com um caracter, para isso usa-se o cifrão, por exemplo, aplicando a expressão `.*br$` o resultado é ``maria_eduarda@hotmail.com.br``, porque é o unico endereço que termina com br.


#### Exemplo
Dado os CEP e suas possíveis estruturas, qual é a REGEX que podemos utilizar?

```
04567003
04567-003
04.567-003
04.567.003
04 567 003
04567 003
04567.003
```

bom, os primeiros dois caracteres são sempre os mesmos então podemos utilizar `^{2}`. Depois disso pode ter, ou não, espaço ou ponto, então usamos `[. ]?`, note que tem um espaço dentro do colchete, para ser mais descritivo pode-se usar também o `[.\s]?`. Em seguida temos novamente mais três algarismos, portanto usamos um `\d{3}`. Depois disso pode vir ou não, espaço, hífen ou ponto, logo a expressão regular é `[-. ]` ou `[-.\s]`. Por fim, tem mais três algarismos então a REGEX é `\d{3}`.

Juntando tudo a expressão regular que engloba todos os exemplos é: `^{3}[.\s]?\d{3}[-.\s]\d{3}`

# Questões em outras linguas

Vamos analisar as questões nos idiomas inglês e espanhol 

## Espanhol

In [5]:
print(stackoverflow_espanhol['Questão'][5])

<p>Siempre he visto que en <code>JavaScript</code> hay:</p>

<ul>
<li>asignaciones <code>=</code></li>
<li>comparaciones <code>==</code> y <code>===</code></li>
</ul>

<p>Creo entender que <code>==</code> hace algo parecido a comparar el valor de la variable y el <code>===</code> también compara el tipo (como un equals de java). </p>

<p><strong>¿Alguien podría confirmarme este punto y extenderlo?</strong>. Soy javero y el no tipado de javascript a veces me encanta y otras lo odio.</p>

<hr>

<p>¿Cuál es la manera correcta en javascript de comparar <code>undefined</code>, <code>null</code> y otros valores por defecto? </p>

<pre><code>variable == null
variable === null
</code></pre>

<p>¿<strong><code>undefined</code></strong> se usa como cadena de texto o como palabra clave? ¿Cual de las siguientes comparaciones es la correcta para un elemento <code>html</code> sin <code>value</code>? (por ejemplo un label sin contenido)</p>

<pre><code>variable == "undefined"
variable === "undefined"

## Inglês


In [6]:
print(stackoverflow_ingles['Questão'][5])

<p>What is the use of the <code>yield</code> keyword in Python? What does it do?</p>

<p>For example, I'm trying to understand this code<sup><strong>1</strong></sup>:</p>

<pre><code>def _get_child_candidates(self, distance, min_dist, max_dist):
    if self._leftchild and distance - max_dist &lt; self._median:
        yield self._leftchild
    if self._rightchild and distance + max_dist &gt;= self._median:
        yield self._rightchild  
</code></pre>

<p>And this is the caller:</p>

<pre><code>result, candidates = [], [self]
while candidates:
    node = candidates.pop()
    distance = node._get_dist(obj)
    if distance &lt;= max_dist and distance &gt;= min_dist:
        result.extend(node._values)
    candidates.extend(node._get_child_candidates(distance, min_dist, max_dist))
return result
</code></pre>

<p>What happens when the method <code>_get_child_candidates</code> is called?
Is a list returned? A single element? Is it called again? When will subsequent calls stop?</p>

<hr>

<

# Usando REGEX Python

O python tem um módulo nativo para manipular expressões regulares, se chamada `re`

In [7]:
import re

A documentação deste módulo é extremamente detalhada

> https://docs.python.org/pt-br/3/library/re.html

Além disso tem um artigo na própria documentação chamado ["HOW TO REGULAR EXPRESSIONS"](https://docs.python.org/pt-br/3/howto/regex.html#regex-howto) que da vários exemplos e casos onde REGEX são usados.

## Aplicando em questões pt-br

In [8]:
questoes_pt = stackoverflow_portugues['Questão']
questoes_en = stackoverflow_ingles['Questão']
questoes_es = stackoverflow_espanhol['Questão']

O módulo `re` tem várias funções, uma delas que é bastante utilizada é o `findall()` que busca no texto todos os caracteres que respeitam a expressão regular passada no parâmetro.

In [9]:
re.findall(r'<p>', questoes_pt[0])


['<p>', '<p>', '<p>', '<p>']

> **obs:** usamos o `r` na frente das aspas simples da string porque quero usar a string pura. Em alguns casos queremos detectar os caracteres "\n" por exemplo, mas ao escrever uma string `\n` o python irá interpretar como um único caracter que representa a quebra de linha. Para evitar isso, usa-se o `r'\n'` para indicar que queremos a string pura (raw)

Então podemos usar a função `findall()` para localizar todas as tags html na questão. Para isso precisamos achar todos os caracteres que tem `<` e terminam com `>`, mas entre eles podem ter ou não algum texto, para isso usamos `.*?`

In [10]:
re.findall(r'<.*?>', questoes_pt[0])

['<p>',
 '<em>',
 '<a href="http://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_de_embaralhamento_criptogr%C3%A1fico" rel="noreferrer">',
 '</a>',
 '</em>',
 '</p>',
 '<p>',
 '<a href="http://pt.wikipedia.org/wiki/Ataque_de_for%C3%A7a_bruta" rel="noreferrer">',
 '</a>',
 '<em>',
 '<a href="http://pt.wikipedia.org/wiki/Keylogger" rel="noreferrer">',
 '</a>',
 '</em>',
 '<a href="http://pt.wikipedia.org/wiki/Criptoan%C3%A1lise_de_mangueira_de_borracha" rel="noreferrer">',
 '<em>',
 '</em>',
 '</a>',
 '<em>',
 '</em>',
 '</p>',
 '<p>',
 '</p>',
 '<p>',
 '<em>',
 '</em>',
 '</p>']

Depois de localizar as tags html no texto precisamos achar um jeito de remove-las ou substituir. O módulo `re` tem a função `re.sub()`

``re.sub(pattern, repl, string, count=0, flags=0)``

Descrição: 
> Retorna a string obtida substituindo as ocorrências não sobrepostas da extremidade esquerda do padrão pattern na string pela substituição repl. Se o padrão não for encontrado, string será retornado inalterado. repl pode ser uma string ou uma função; se for uma string, qualquer escape de contrabarra será processado. 

In [11]:
print(re.sub(r'<.*?>', '   === TESTE SUB ===  ', questoes_pt[5]))

   === TESTE SUB ===  Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:   === TESTE SUB ===  

   === TESTE SUB ===     === TESTE SUB ===  //----CONSULTA SQL----//
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');
   === TESTE SUB ===     === TESTE SUB ===  

   === TESTE SUB ===  Logo, digamos que o usuário usar a sentença:    === TESTE SUB ===  1); DROP TABLE Produtos;   === TESTE SUB ===   para ao campo    === TESTE SUB ===  valor   === TESTE SUB ===   o comando ficaria:    === TESTE SUB ===  

   === TESTE SUB ===     === TESTE SUB ===  insert into Produtos (coluna) values(1); DROP TABLE Produtos;
   === TESTE SUB ===     === TESTE SUB ===  

   === TEST

Note que onde esta o `== TESTE SUB ===` era onde estavam as tags html e a função substituiu. Isto é, para a função ``re.sub(pattern, repl, string, count=0, flags=0)`` passamos o `pattern` que é o padrão regex, o `repl` que é o que será substituido (replace) e a string onde será substituitdo. Então para remover as tags basta colocar um espaço vazio, verifique:

In [12]:
print(re.sub(r'<.*?>', '', questoes_pt[5]))

Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:

//----CONSULTA SQL----//
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');


Logo, digamos que o usuário usar a sentença: 1); DROP TABLE Produtos; para ao campo valor o comando ficaria: 

insert into Produtos (coluna) values(1); DROP TABLE Produtos;


Ele vai inserir um novo registro cujo o campo coluna será 1 e logo em seguida ele vai deletar a tabela Produtos.

Como posso melhorar meu código para prevenir essa situação?



### Desempenho

Existem duas formas de construir uma expressão regular com o `re`. Atraves da passagem da expressão regular, igual fizemos antes:

```python
re.findall(r'{minha_expressao_regular}', string_a_ser_filtrada)
```

mas também podemos criar um objeto, que compila a regex e ai só passamos a variável que queremos filtrar. Por exemplo: 

```python
regex = re.compile(r'{minha_expressao_regular}')
regex.findall(string_a_ser_filtrada)
```

Entretanto, qual a diferença entre essas duas formas? Para comparar vamos usar um outro módulo chamado `timeit` nativo do python para calcular o desempenho de uma execução:


In [13]:
from timeit import timeit

vamos criar a expressão regular `r'70'` que procura o número setenta em `12345712365123954623469843062370`, ai basicamente passamos um código a ser executado dentro de uma docstring `("""codigo""")`

In [14]:
setup = """import re"""
timeit("""re.search(r'70', '12345712365123954623469843062370')""", setup)

0.8894951949914685

o tempo de execução do código `re.search(r'70', '12345712365123954623469843062370')` foi de 0.94 segundos. Obviamente não é uma única execução, o módulo timeit executa função 1 milhão de vezes.
 
Agora vamos fazer algo parecido, mas criando um objeto do tipo regex.

In [15]:
setup = """import re
regex = re.compile(r'70')"""

timeit("""regex.search('12345712365123954623469843062370')""", setup)

0.2117444889881881

Note que o tempo foi 4x menor para 1 milhão de execuções. Outra forma de analisar isso é usando o `%%time` do próprio notebook para mensurar o tempo de execução de uma célula:

In [16]:
%%time
for i in range(10**6):
    re.search(r'70', '12345712365123954623469843062370')

CPU times: user 1.06 s, sys: 0 ns, total: 1.06 s
Wall time: 1.06 s


In [17]:
regex = re.compile(r'70')

In [18]:
%%time
for i in range(10**6):
    regex.search('12345712365123954623469843062370')

CPU times: user 250 ms, sys: 50 µs, total: 250 ms
Wall time: 249 ms


In [19]:
1.04 / (252*10**(-3))

4.126984126984127

novamente o resultado é na mesma ordem de grandeza.

Concluimos então que criar uma expressão regular (regex) usando o `compile()` tem um desempenho muito maior, por isso é extremamente recomendado criar um objeto em vez de passar a expressão regular como parâmetro de um método do módulo `re`.

### Removendo tags html 

In [20]:
def filtro_regex(textos, regex):
    """
    Aplica regex em um texto ou em uma estrutura de dados com textos.
    Retorna um texto filtrado ou a estrutura de dados com textos filtrados.
    """
    
    if type(textos) == str:
        return regex.sub('', textos)
    elif type(textos) in (list, tuple, set, np.array, pd.Series, pd.DataFrame):
        return pd.Series([regex.sub('', texto) for texto in textos])
    else:
        raise Exception('Não foi possível aplicar o REGEX')
    

criando `regex`: 

In [21]:
remove_html = re.compile(r'<.*?>')

testando:

In [22]:
print('Com tags HTML')
display(questoes_pt[:5])

print('Sem tags HTML')
display(filtro_regex(questoes_pt[:5], remove_html))

Com tags HTML


0    <p>Se eu fizer o <em><a href="http://pt.wikipe...
1    <p>Qual é a diferença entre <code>INNER JOIN</...
2    <p>Uma dúvida muito comum é por que devemos pa...
3    <p>É comum encontrar uma mensagem de erro que ...
4    <p>Me parecem termos muito próximos e eventual...
Name: Questão, dtype: object

Sem tags HTML


0    Se eu fizer o hash de senhas antes de armazená...
1    Qual é a diferença entre INNER JOIN e OUTER JO...
2    Uma dúvida muito comum é por que devemos parar...
3    É comum encontrar uma mensagem de erro que diz...
4    Me parecem termos muito próximos e eventualmen...
dtype: object

### Removendo os códigos,

Note que temos alguns códigos nos textos:

In [23]:
print(filtro_regex(questoes_en[5], remove_html))

What is the use of the yield keyword in Python? What does it do?

For example, I'm trying to understand this code1:

def _get_child_candidates(self, distance, min_dist, max_dist):
    if self._leftchild and distance - max_dist &lt; self._median:
        yield self._leftchild
    if self._rightchild and distance + max_dist &gt;= self._median:
        yield self._rightchild  


And this is the caller:

result, candidates = [], [self]
while candidates:
    node = candidates.pop()
    distance = node._get_dist(obj)
    if distance &lt;= max_dist and distance &gt;= min_dist:
        result.extend(node._values)
    candidates.extend(node._get_child_candidates(distance, min_dist, max_dist))
return result


What happens when the method _get_child_candidates is called?
Is a list returned? A single element? Is it called again? When will subsequent calls stop?




1. This piece of code was written by Jochen Schulz (jrschulz), who made a great Python library for metric spaces. This is the link to 

O que precisamos fazer é remover esses trechos de código porque dado que a maioria das linguagens de programação são escritas em inglês o modelo pode identificar que o idioma do texto é inglês apenas porquê tem trechos de código.

Observando o texto puto vemos que tem uma tag HTML chamada `<code> CÓDIGO AQUI </code>`, podemos aplicar uma expressão regular para remover todos os textos que esta dentro desta tag:

In [24]:
# começa com "<code>" tem um grupo de qualquer caracter e quebras de linha e termina com </code>
remove_linguagem_programacao = re.compile(r'<code>(.|(\n))*?</code>')

print(questoes_en[5])
print(20*'=')
print(remove_linguagem_programacao.sub('CODIGO AQUI',questoes_en[5]))

<p>What is the use of the <code>yield</code> keyword in Python? What does it do?</p>

<p>For example, I'm trying to understand this code<sup><strong>1</strong></sup>:</p>

<pre><code>def _get_child_candidates(self, distance, min_dist, max_dist):
    if self._leftchild and distance - max_dist &lt; self._median:
        yield self._leftchild
    if self._rightchild and distance + max_dist &gt;= self._median:
        yield self._rightchild  
</code></pre>

<p>And this is the caller:</p>

<pre><code>result, candidates = [], [self]
while candidates:
    node = candidates.pop()
    distance = node._get_dist(obj)
    if distance &lt;= max_dist and distance &gt;= min_dist:
        result.extend(node._values)
    candidates.extend(node._get_child_candidates(distance, min_dist, max_dist))
return result
</code></pre>

<p>What happens when the method <code>_get_child_candidates</code> is called?
Is a list returned? A single element? Is it called again? When will subsequent calls stop?</p>

<hr>

<

Como foi visto o código foi substituido por `CODIGO AQUI`, vamos criar uma função que remove os código: 

In [25]:
print('Com tags CODE')
display(questoes_en[3:6])

print('Sem tags CODE')
display(filtro_regex(questoes_en[3:6], remove_linguagem_programacao))

Com tags CODE


3    <blockquote>\n  <p><strong>Moderator Note:</st...
4    <p>I've been messing around with <a href="http...
5    <p>What is the use of the <code>yield</code> k...
Name: Questão, dtype: object

Sem tags CODE


0    <blockquote>\n  <p><strong>Moderator Note:</st...
1    <p>I've been messing around with <a href="http...
2    <p>What is the use of the  keyword in Python? ...
dtype: object

### Aplicando filtragens 

Vamos aplicar a filtragem que retira os códigos das linguagens de programação das perguntas e em seguida remover as tags HTML. Não podemo remover primeiro o HTML e depois os trechos de código, porque o que identifica o código é justamente a tag HTML

In [26]:
def filtro_codigos_html(serie):
    """
    Aplica um filtro pra retirar os códigos das linguagens
    em seguida outro filtro pra remover tags HTML
    """
    serie = filtro_regex(serie, remove_linguagem_programacao)
    serie = filtro_regex(serie, remove_html)
    return serie

In [27]:
stackoverflow_portugues['questao_sem_code_tag'] = filtro_codigos_html(questoes_pt)
stackoverflow_espanhol['questao_sem_code_tag'] = filtro_codigos_html(questoes_es)
stackoverflow_ingles['questao_sem_code_tag'] = filtro_codigos_html(questoes_en)

In [28]:
print(25*'_', 'QUESTÃO PT-BR', 25*'_','\n')
print(stackoverflow_portugues.questao_sem_code_tag[0],'\n')
print(60*'_','\n')
print(25*'_', 'QUESTÃO ES', 25*'_','\n')
print(stackoverflow_espanhol.questao_sem_code_tag[0],'\n')
print(60*'_','\n')
print(25*'_', 'QUESTÃO EN-EUA', 25*'_','\n')
print(stackoverflow_ingles.questao_sem_code_tag[0], '\n')
print(60*'_','\n')

_________________________ QUESTÃO PT-BR _________________________ 

Se eu fizer o hash de senhas antes de armazená-las em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém?

Estou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque, como força bruta na página de login da aplicação, keylogger no cliente e criptoanálise rubberhose. Qualquer forma de hash não vai impedir esses ataques.

Tenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido. Como dar maior garantia de segurança neste aspecto?

Quais preocupações adicionais evitariam o acesso às senhas? Existem formas melhores de fazer esse hash?
 

____________________________________________________________ 

_________________________ QUESTÃO ES _________________________ 

Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto (strings) y en las que se insertan/concaten

Note que agora temos as questões sem trechos de códigos e também sem tags, mas falta uma outra modificação: *retirar pontuações e caracteres especiais*, por exemplo: `? , ñ !`

### Excluindo caracteres especiais

Existe um meta caracter expresso pelo `^` que indica o complementar, por exemplo, `[^5]` irá corresponder a qualquer caractere exceto `'5'`, e `[^^]` irá corresponder a qualquer caractere exceto `'^'`. 

In [62]:
remove_pontuacao = re.compile(r'[^\s\w]')

In [63]:
print(stackoverflow_espanhol.questao_sem_code_tag[0])

print(remove_pontuacao.sub('',stackoverflow_espanhol.questao_sem_code_tag[0]))

Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto (strings) y en las que se insertan/concatenan valores obtenidos de alguna fuente (normalmente proveniente del usuario), lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas, como por ejemplo:



Eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación (web o no) porque si el usuario introdujese un valor como  nos encontraríamos con que la sentencia ejecutada sería:



Y se eliminaría la tabla Usuarios con todos los datos contenidos en ella. 

¿Cómo puedo evitar que la inyección SQL ocurra en PHP?

Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto strings y en las que se insertanconcatenan valores obtenidos de alguna fuente normalmente proveniente del usuario lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas como por ejemplo



Eso es un ejemplo de una vulnerabilidad grave en la seguridad de 

Através da regex `[^\s\w]`, isto é "*remova tudo que não for \s (espaço) e alphanumérico*", agora precisamos de uma função que faça isso:

In [64]:
def filtro_codigos_html(serie):
    """
    Aplica um filtro pra retirar os códigos das linguagens
    em seguida outro filtro pra remover tags HTML
    """
    serie = filtro_regex(serie, remove_linguagem_programacao)
    serie = filtro_regex(serie, remove_html)
    serie = filtro_regex(serie, remove_pontuacao)
    return serie

In [65]:
stackoverflow_portugues['questao_sem_code_tag'] = filtro_codigos_html(questoes_pt)
stackoverflow_espanhol['questao_sem_code_tag'] = filtro_codigos_html(questoes_es)
stackoverflow_ingles['questao_sem_code_tag'] = filtro_codigos_html(questoes_en)

In [67]:
print(stackoverflow_espanhol.questao_sem_code_tag[0])

Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto strings y en las que se insertanconcatenan valores obtenidos de alguna fuente normalmente proveniente del usuario lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas como por ejemplo



Eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación web o no porque si el usuario introdujese un valor como  nos encontraríamos con que la sentencia ejecutada sería



Y se eliminaría la tabla Usuarios con todos los datos contenidos en ella 

Cómo puedo evitar que la inyección SQL ocurra en PHP



### Deixando tudo em minusculo

In [92]:
def minusculo(textos):
    if type(textos) == str:
        return textos.lower()
    elif type(textos) in (list, tuple, np.array, pd.Series, pd.DataFrame):
        if type(textos[0]) == str:
            return pd.Series([texto.lower() for texto in textos])
        else:
            raise Exception('Não foi possível aplicar .lower()')
    else:
        raise Exception('Não foi possível identificar o tipo')
    

In [95]:
display(minusculo(stackoverflow_portugues.questao_sem_code_tag))
display(minusculo(stackoverflow_espanhol.questao_sem_code_tag))
display(minusculo(stackoverflow_ingles.questao_sem_code_tag))

0      se eu fizer o hash de senhas antes de armazená...
1      qual é a diferença entre  e  podem me dar algu...
2      uma dúvida muito comum é por que devemos parar...
3      é comum encontrar uma mensagem de erro que diz...
4      me parecem termos muito próximos e eventualmen...
                             ...                        
495    já vi esse termo polyfill sendo utilizado vári...
496    esses dias me deparei com um trecho de um sql ...
497    por vezes vejo em documentações ou especificaç...
498    na especificação do protocolo http mais precis...
499    eu preciso fazer o upload de uma foto e em seg...
Length: 500, dtype: object

0      las sentencias dinámicas son sentencias sql qu...
1      unas veces los cálculos funcionan correctament...
2      cuál es la diferencia entre un  y un \n\ny cuá...
3      la mayoría de nosotros decimos muchas veces si...
4      pregunta\n\ncómo validar un email que acepte t...
                             ...                        
495    para escribir una respuesta a 191c243mo funcio...
496    estoy empezando a estudiar e investigar un poc...
497    estoy empezando con c y tengo la siguiente dud...
498    cuál es la diferencia entre interfaz y clase a...
499    necesito agregarle un estilo a mi checkbox par...
Length: 500, dtype: object

0      here is a piece of c code that seems very pecu...
1      i accidentally committed the wrong files to gi...
2      i want to delete a branch both locally and rem...
3      \n  moderator note given that this question ha...
4      ive been messing around with json for some tim...
                             ...                        
495    i just installed  and  for additional modules\...
496    this came to my mind after i learned the follo...
497    is it possible to set up a basic html page to ...
498    i have a repository in git i made a branch the...
499    i came across some java code that had the foll...
Length: 500, dtype: object

In [96]:
def filtro_codigos_html(serie):
    """
    Aplica um filtro pra retirar os códigos das linguagens
    em seguida outro filtro pra remover tags HTML
    """
    serie = filtro_regex(serie, remove_linguagem_programacao)
    serie = filtro_regex(serie, remove_html)
    serie = filtro_regex(serie, remove_pontuacao)
    serie = minusculo(serie)
    return serie

In [97]:
stackoverflow_portugues['questao_sem_code_tag'] = filtro_codigos_html(questoes_pt)
stackoverflow_espanhol['questao_sem_code_tag'] = filtro_codigos_html(questoes_es)
stackoverflow_ingles['questao_sem_code_tag'] = filtro_codigos_html(questoes_en)

In [98]:
display(stackoverflow_portugues.questao_sem_code_tag)
display(stackoverflow_espanhol.questao_sem_code_tag)
display(stackoverflow_ingles.questao_sem_code_tag)

0      se eu fizer o hash de senhas antes de armazená...
1      qual é a diferença entre  e  podem me dar algu...
2      uma dúvida muito comum é por que devemos parar...
3      é comum encontrar uma mensagem de erro que diz...
4      me parecem termos muito próximos e eventualmen...
                             ...                        
495    já vi esse termo polyfill sendo utilizado vári...
496    esses dias me deparei com um trecho de um sql ...
497    por vezes vejo em documentações ou especificaç...
498    na especificação do protocolo http mais precis...
499    eu preciso fazer o upload de uma foto e em seg...
Name: questao_sem_code_tag, Length: 500, dtype: object

0      las sentencias dinámicas son sentencias sql qu...
1      unas veces los cálculos funcionan correctament...
2      cuál es la diferencia entre un  y un \n\ny cuá...
3      la mayoría de nosotros decimos muchas veces si...
4      pregunta\n\ncómo validar un email que acepte t...
                             ...                        
495    para escribir una respuesta a 191c243mo funcio...
496    estoy empezando a estudiar e investigar un poc...
497    estoy empezando con c y tengo la siguiente dud...
498    cuál es la diferencia entre interfaz y clase a...
499    necesito agregarle un estilo a mi checkbox par...
Name: questao_sem_code_tag, Length: 500, dtype: object

0      here is a piece of c code that seems very pecu...
1      i accidentally committed the wrong files to gi...
2      i want to delete a branch both locally and rem...
3      \n  moderator note given that this question ha...
4      ive been messing around with json for some tim...
                             ...                        
495    i just installed  and  for additional modules\...
496    this came to my mind after i learned the follo...
497    is it possible to set up a basic html page to ...
498    i have a repository in git i made a branch the...
499    i came across some java code that had the foll...
Name: questao_sem_code_tag, Length: 500, dtype: object